In [1]:
import numpy as np
import helpers_and_variables as hlp
import ML_helpers as ml_hlp
import shap
import random

import pandas as pd
# import pixiedust
# %%pixie_debugger

In [ ]:
"""
read the row data as pd data frame
"""
dataPath = "C:/Users/a7mad/Desktop/MEX/PekLung/PekLUng_20210503sav.xlsx"
password =  "d46cf574-84e1-11ec-a8a3-0242ac120002"#
rawdataDF = hlp.get_exL_df(stringPath=dataPath, password=password, sheetNum=1)
""" 
read the data info as pd data frame
"""
dataInfoPath = "C:/Users/a7mad/Desktop/MEX/PekLung/Datainformation_minroAdjusted.xlsx"
dataInfoDF = hlp.get_cleaned_dataInfo_df(dataInfoPath)
katInfoDF = hlp.get_cleaned_katInfo_df(dataInfoPath)
"""
get dictionary of data information from data info data fram
"""
dict_of_katInfo = hlp.get_dict_of_katInfoDF(katInfoDF)
dict_of_dataInfo = hlp.get_dict_of_dataInfoDF(dataInfoDF)
""" 
Get the labels, under name Lungcancer_Num
check whether labels are 1=yes LC or 2=No LC and check STUDY_1 if valid, remove unlabeled and invalid patients

"""
rawdataDF, labels, removed_indices = hlp.get_labels_and_indices_unlabeled_patients(rawdataDF)
"""
Remove features, which includes information about the label, like diagnos2, aslo modules names
DiagnosticInvestigation (need to be discussed, since it includes dignostic which means non early prediction).
columns_tobe_removed=None --> predefind columns will be removed, see the function in helpers_and_variables file.
remove_cols_with_dates=True --> removes all columns with dates(this is relevance in case of tfidf), 
otherwise consider using converting dates to days, see function hlp.get_dates_in_days() in next cell.
"""
rawdataDF =  hlp.get_dataframe_without_cols(rawdataDF,  columns_tobe_removed=None, remove_cols_with_dates=False)

copy_rawdata = pd.DataFrame.copy(rawdataDF)

In [ ]:
copy_rawdata = hlp.get_dates_in_days(copy_rawdata);

In [4]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(katInfoDF)
# copy_rawdata

In [5]:
# list_of_vars = katInfoDF['Variable'].tolist()
# list_of_labels = katInfoDF['Label'].tolist()
# list_of_values = katInfoDF['Value'].tolist()

# list_of_questions = dataInfoDF['Label'].tolist()
# list_of_quesNams = dataInfoDF['Variable'].tolist()

# cleanedListOfStrings = hlp.get_cleaned_list_of_strings(list_of_questions, stemm=True)
# cleanedListOfStrings
# cleanedWordsArray = hlp.get_array_of_words(cleanedListOfStrings)
"""
get a list with qustions and answers for one patient
""";
# list_ques_answers = list(main_dict['1001'].values())
# list_cleaned_ques_answers = hlp.get_cleaned_list_of_strings(list_ques_answers,stemm=False)
# token_words_array = hlp.get_array_of_words(list_cleaned_ques_answers)
# unique_token_words = np.unique(token_words_array, return_counts=True)
# stemmed_by_nltk = hlp.get_stemmed_strings_as_nltk_SnowballStemmer(list_of_questions, ignore_stopwords=False)
# tokenized_by_nltk =  hlp.get_tokenized_strings_by_nltk(cleanedListOfStrings)

In [6]:
# cols_with_dates = hlp.get_cols_with_dates(copy_rawdata)
# df_dates_before = pd.DataFrame(columns=cols_with_dates)
# df_dates_after = pd.DataFrame(columns=cols_with_dates)
# for column in cols_with_dates:
#     df_dates_before[column] = rawdataDF[column]
#     df_dates_after[column] = copy_rawdata[column]

In [7]:
# copy_rawdata.to_csv('C:/Users/a7mad/Desktop/MEX/PekLung/dates_adjusted.csv')
# copy_rawdata.to_excel('C:/Users/a7mad/Desktop/MEX/PekLung/dates_adjusted_excel.xlsx')

# x = pd.read_csv('dates_adjusted.csv')
# x = pd.read_excel('dates_adjusted_excel.xlsx')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_dates_before)
# display(df_dates_before)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):    
#     display(df_dates_after)

In [9]:
"""
get a dictionary of each patient according to this format:
Patient number: {questions name, the questions text, corresponding answers}
If clear_missings_or_Non true, it clears questions (features),
where answeres are missing or the answers is no/missing.
amount_data = None --> all data is considered
"""
main_dict, ind_num_removed_features = hlp.get_dict_of_questions_answers(copy_rawdata, 
                                                                        dataInfoDF, 
                                                                        katInfoDF, 
                                                                        amount_data = None,
                                                                        clear_missings_or_Non=True,
                                                                        clear_ques_with_negative_answeres=False)
"""
check how many features were removed by get_dict_of_questions_answers()

"""
max_num_feature_removed = 0
ind_of_max_feature_removed = 0

for ind, count in ind_num_removed_features:
    if count > max_num_feature_removed:
        max_num_feature_removed = count
        ind_of_max_feature_removed = ind
print("Most number of features removed from a one patient is: ", max_num_feature_removed)
print("Patient index with most removed features is: ", ind_of_max_feature_removed)
print("Patient number or id: {0}, with remaining number of features: {1}".format(str(int(rawdataDF.loc[ind_of_max_feature_removed][0])), len(main_dict[str(int(rawdataDF.loc[ind_of_max_feature_removed][0]))])))
print("\nTotal length of the main dictionary, number of patients: ", len(main_dict))
#main_dict[str(int(rawdataDF.loc[ind_of_max_feature_removed][0]))]

Most number of features removed from a one patient is:  566
Patient index with most removed features is:  10
Patient number or id: 1013, with remaining number of features: 113

Total length of the main dictionary, number of patients:  506


In [10]:
"""
In case you need to write/save the dictionary
"""
save_dict_path = "C:/Users/a7mad/Desktop/MEX/PekLung/dict"
# save_dict_path = "C:/Users/a7mad/Desktop/MEX/PekLung/dict_nodates"
# save_dict_path = "C:/Users/a7mad/Desktop/MEX/PekLung/dict_removed_nos_nodates"
# labels_path = "C:/Users/a7mad/Desktop/MEX/PekLung/labels"

hlp.write_dict_as_json_file(main_dict, file_path=save_dict_path)
# hlp.write_list_as_json_file(labels, file_path=labels_path)
js = hlp.load_dict_from_json_file(file_path=save_dict_path)
# lbl = hlp.load_list_from_json_file(file_path=labels_path)
# print(lbl == labels)
print(js==main_dict)

True


In [ ]:
# data_list = hlp.get_data_list_from_main_dict(main_dict, stemm=True)
# data_list
# vectorizer = ml_hlp.get_tfidf_vectorization_model(data_list)
# X_train_vectorized = ml_hlp.get_tfidf_vectorized_data(tfidf_model=vectorizer,
#                                                       to_be_vectorized_data=data_list)
# X_train = ml_hlp.get_csr_matrix(X_train_vectorized)
# main_dict['1001']